# 1.0 Instalando o Plotly e Dash e importando Bibliotecas

In [1]:
# Install Plotly
!pip install Plotly==4.12

# Install Dash
!pip install dash
!pip install dash-html-components
!pip install dash-core-components
!pip install dash-table

     |████████████████████████████████| 13.1MB 309kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
     |████████████████████████████████| 81kB 3.8MB/s 
     |████████████████████████████████| 1.0MB 8.6MB/s 
     |████████████████████████████████| 3.5MB 43.0MB/s 
     |████████████████████████████████| 194kB 45.2MB/s 
     |████████████████████████████████| 1.8MB 39.9MB/s 
     |████████████████████████████████| 358kB 42.1MB/s 
  Created wheel for dash: filename=dash-1.17.0-cp36-none-any.whl size=83728 sha256=8f7f75614936ffae8dd9d3062c87e5bd60dacdfce967334414d6bda73c9a9120
  Stored in directory: /root/.cache/pip/wheels/e0/32/13/8a569b638a79cf80a4a721f56040d91198adec28e3bc8123fc
  Created wheel for dash-renderer: filename=dash_renderer-1.8.3-cp36-none-any.whl size=1013944 sha256=787455af7b50fe0fa55c51fdad56a88b1357eee3c3bb21b4dd7bf533b079c9f0
  Stored in directory: /root/.cache/pip/wheels/cf/2b/5f/0928a6d1b7ebf2

In [2]:
import os.path
import sys, json
import requests
import subprocess

import numpy as np
import pandas as pd
import plotly.express as px

from requests.exceptions import RequestException
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

from collections import namedtuple

# 2.0 Configurando o Ngrok

In [3]:
def download_ngrok():
    if not os.path.isfile('ngrok'):
        !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
        !unzip -o ngrok-stable-linux-amd64.zip
    pass

In [4]:
Response = namedtuple('Response', ['url', 'error'])

def get_tunnel():
    try:
        Tunnel = subprocess.Popen(['./ngrok','http','8050'])

        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)

        res = session.get('http://localhost:4040/api/tunnels')
        res.raise_for_status()

        tunnel_str = res.text
        tunnel_cfg = json.loads(tunnel_str)
        tunnel_url = tunnel_cfg['tunnels'][0]['public_url']

        return Response(url=tunnel_url, error=None)
    except RequestException as e:
        return Response(url=None, error=str(e))

# 3.0 O Dash apps

Visualizando os dados do nosso dataframe

In [5]:
import pandas as pd

dadosInpe48h = 'http://queimadas.dgi.inpe.br/queimadas/dados-abertos/download/?id=focos_brasil&time=48h' 

df = pd.read_csv(dadosInpe48h)

df.head()

,FID,id,longitude,latitude,data_hora_gmt,satelite,municipio,estado,pais,municipio_id,estado_id,pais_id,numero_dias_sem_chuva,precipitacao,risco_fogo,bioma,geom
0,focos_brasil_ontem_hoje_ref.28d758ec-ef68-384a...,28d758ec-ef68-384a-8670-0b17def7f611,-49.840,-3.527,2020-11-23T17:00:00,AQUA_M-T,PACAJÁ,PARÁ,Brasil,1505486,15,33,NaN,NaN,NaN,Amazônia,POINT (-3.527 -49.84)
1,focos_brasil_ontem_hoje_ref.5c57cd1d-b62b-3398...,5c57cd1d-b62b-3398-a15c-14c74c872a3c,-56.087,-1.731,2020-11-23T17:00:00,AQUA_M-T,ORIXIMINÁ,PARÁ,Brasil,1505304,15,33,NaN,NaN,NaN,Amazônia,POINT (-1.731 -56.087)
2,focos_brasil_ontem_hoje_ref.60c4de01-1bbd-3cb8...,60c4de01-1bbd-3cb8-82c1-34237dc121e1,-49.739,-3.457,2020-11-23T17:00:00,AQUA_M-T,BAIÃO,PARÁ,Brasil,1501204,15,33,NaN,NaN,NaN,Amazônia,POINT (-3.457 -49.739)
3,focos_brasil_ontem_hoje_ref.27955fff-63d3-3e73...,27955fff-63d3-3e73-a55e-55a98d9ad5e5,-50.530,-3.589,2020-11-23T17:00:00,AQUA_M-T,PACAJÁ,PARÁ,Brasil,1505486,15,33,NaN,NaN,NaN,Amazônia,POINT (-3.589 -50.53)
4,focos_brasil_ontem_hoje_ref.4b160504-97df-3e1d...,4b160504-97df-3e1d-ace7-5cd66979365d,-49.730,-3.455,2020-11-23T17:00:00,AQUA_M-T,BAIÃO,PARÁ,Brasil,1501204,15,33,NaN,NaN,NaN,Amazônia,POINT (-3.455 -49.73)


Código da aplicação com o Dash app

In [10]:
%%writefile my_dash_app.py
import dash
import plotly.graph_objs as go
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd 

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

dadosInpe48h = 'http://queimadas.dgi.inpe.br/queimadas/dados-abertos/download/?id=focos_brasil&time=48h' 

df = pd.read_csv(dadosInpe48h)

token = open("/content/token.txt").read()

fig = px.scatter_mapbox(df, 
                        lat='latitude', 
                        lon='longitude',
                        color='bioma',
                       
                        zoom=3.8,
                        hover_data=['municipio', 'estado', 'risco_fogo'],)

fig.update_layout(mapbox_style="dark", mapbox_accesstoken=token,)
fig.update_layout(margin={'r':0,'t':0,'l':0,'b':0}, paper_bgcolor="#1C1C1C", font={"color":"White"})


fig2 = px.histogram (df,
                     x = 'estado',
                     color = 'bioma',
                     
                     title='Os tipos de biomas',
                     labels={'estado':'Estados'},
                     )

fig2.update_layout(paper_bgcolor="#1C1C1C", font={"color":"White"}, plot_bgcolor='#363636')

app.layout = html.Div( children=[
    dcc.Graph( 
        id='life-exp-vs-gd',
        figure=fig
    ),  html.Div(
                children=[],
              ),
              dcc.Graph(id="histograma", figure=fig2),
])

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Overwriting my_dash_app.py


Run the dash app:

In [7]:
download_ngrok()

--2020-11-24 01:15:36--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.224.51.10, 52.203.100.2, 34.194.108.77, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.224.51.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  6.60MB/s    in 2.0s    

2020-11-24 01:15:39 (6.60 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
retorno = get_tunnel()
print(retorno)
!python my_dash_app.py

# Referências 

* [Video: Tableros Dash en Colab y tuneles con ngrok](https://www.youtube.com/watch?v=g6M3mAHFcyU)
*   [ngrok](https://ngrok.com/)
*   [Dash Plotly](https://dash.plotly.com/)
*   [Interactive Graphing](https://dash.plotly.com/interactive-graphing)
*   [Dash for Beginners - DataCamp](https://www.datacamp.com/community/tutorials/learn-build-dash-python)
